<a href="https://colab.research.google.com/github/CastHash532/kaggle-automl/blob/main/Notebooks/Kaggle-News-NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flaml[notebook];

     |████████████████████████████████| 155 kB 24.4 MB/s 
     |████████████████████████████████| 67.4 MB 34 kB/s 
     |████████████████████████████████| 2.0 MB 68.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
     |████████████████████████████████| 158 kB 75.1 MB/s 
     |████████████████████████████████| 12.4 MB 170 kB/s 
     |████████████████████████████████| 757 kB 44.9 MB/s 
  Created wheel for openml: filename=openml-0.10.2-py3-none-any.whl size=190318 sha256=201ec69c30adc2146b3d1138e73345797989ad66ac3804c08d3e41e9f4618253
  Stored in directory: /root/.cache/pip/wheels/9c/9e/f3/6a5ebf16527d7fe22d9bc1652bc9beb5dc9fcfdeb75e805400
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11731 sha256=3d04c7492017209d50b8c3ab05451b84cf90003b96cdb085abe2041309f8c4b5
  Stored in directory: /root/.cache/pip/wheels/1f/0f/15/332ca86cbebf25ddf98518caaf887945fbe1712b97a0f2493b
Successfully built openml liac-arff
  Attempting uninstall: scikit-learn


In [2]:
!pip install autokeras

     |████████████████████████████████| 166 kB 31.5 MB/s 
     |████████████████████████████████| 454.3 MB 15 kB/s 
     |████████████████████████████████| 97 kB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 46.4 MB/s 
     |████████████████████████████████| 462 kB 72.5 MB/s 
     |████████████████████████████████| 1.2 MB 51.6 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.40.0
    Uninstalling grpcio-1.40.0:
      Successfully uninstalled grpcio-1.40.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0


In [3]:
exit()

## Authenticating with Kaggle using kaggle.json

Navigate to https://www.kaggle.com. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account) and select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below to upload kaggle.json to your Colab runtime.

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


## Load data and preprocess



In [2]:
!kaggle datasets download -d rmisra/news-category-dataset

news-category-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!ls

auto_model		       news-category-dataset.zip
News_Category_Dataset_v2.json  sample_data


In [4]:
!unzip news-category-dataset.zip

Archive:  news-category-dataset.zip
replace News_Category_Dataset_v2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: News_Category_Dataset_v2.json  


In [5]:
import pandas as pd
import numpy as np

dataset = pd.read_json('/content/News_Category_Dataset_v2.json',lines=True)

In [6]:
from sklearn.model_selection import train_test_split

ds_train, ds_test = train_test_split(dataset,test_size=0.2, random_state=42)


In [18]:
ds_train = ds_train[:5000]
ds_test = ds_test[:2000]

In [ ]:
ds_train

## Autokeras

In [19]:
import os

import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files

import autokeras as ak

In [20]:
# Initialize the text classifier.
clf = ak.TextClassifier(
    overwrite=True, max_trials=2
)  # It only tries 1 model as a quick demo.

In [25]:
# Feed the text classifier with training data.
clf.fit(pd.Series.to_numpy(ds_train['headline']), ds_train['category'], epochs=3)

Trial 3 Complete [00h 00m 49s]
val_loss: 3.2920169830322266

Best val_loss So Far: 3.0174143314361572
Total elapsed time: 00h 03m 15s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
157/157 [==============================] - 32s 202ms/step - loss: 3.3506 - accuracy: 0.1570
Epoch 2/3
157/157 [==============================] - 32s 204ms/step - loss: 3.0983 - accuracy: 0.2208
Epoch 3/3
157/157 [==============================] - 32s 203ms/step - loss: 2.8064 - accuracy: 0.2790
INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets


In [26]:
# Predict with the best model.
predicted_y = clf.predict(pd.Series.to_numpy(ds_test['headline']))
# Evaluate the best model with testing data.
print(clf.evaluate(pd.Series.to_numpy(ds_test['headline']), ds_test['category']))

63/63 [==============================] - 4s 54ms/step - loss: 2.9524 - accuracy: 0.2700
[2.9524099826812744, 0.27000001072883606]
